In [ ]:
!pip install transformers
!pip install accelerate
!pip install einops
!pip install bitsandbytes
!pip install langchain

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "MBZUAI/LaMini-T5-738M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, device_map = "auto", torch_dtype = torch.float32)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [6]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

pipeline = pipeline(
  "text2text-generation",
  model = model,
  tokenizer = tokenizer,
  max_length = 256,
  do_sample = True,
  temperature = 0.3,
  top_p = 0.95
)

llm = HuggingFacePipeline(pipeline = pipeline)

In [7]:
prompt = "Write me a section title for deploying a model on SageMaker and generated an endpoint"

In [8]:
generated_text = llm(prompt)
generated_text

'"SageMaker Model Deployment and Endpoint Generation"'

In [9]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
    'HF_TASK': 'text2text-generation',
    'device_map': 'auto',
    'torch_dtype': 'torch.float32'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.xlarge",
  )
  
# send request
predictor.predict({
	"inputs": "Write a short article on Blockchain?",
})

------------!

[{'generated_text': 'Blockchain is a decentralized digital ledger that records transactions in a secure and transparent manner.'}]

In [12]:
endpoint_name = predictor.endpoint_name
print(endpoint_name)

huggingface-pytorch-tgi-inference-2023-07-16-14-08-59-732


In [27]:
prompt = "Write an article on Large Language Models"

# hyperparameter payload
payload = {
    "inputs": prompt,
    "parameters": {
        "do_sample": True,
        "top_p": 0.7,
        "temperature": 0.3,
        "top_k": 50,
        "max_new_tokens": 512,
        "repetition_penalty": 1.03
    }
}

runtime = boto3.client("runtime.sagemaker")
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType="application/json", Body= json.dumps(payload))

In [25]:
response

{'ResponseMetadata': {'RequestId': '9610fc4e-1dc7-4fd6-b909-ef1692c5d8a8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9610fc4e-1dc7-4fd6-b909-ef1692c5d8a8',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Sun, 16 Jul 2023 14:31:19 GMT',
   'content-type': 'application/json',
   'content-length': '672',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f67434741c0>}

In [28]:
prediction = json.loads(response["Body"].read().decode("utf-8"))

In [29]:
prediction

[{'generated_text': 'Large Language Models (LLMs) are a type of machine learning algorithm that is used to analyze large amounts of text data. These models are designed to recognize patterns and make predictions based on the linguistic features of the text. LLMs are widely used in various applications, including speech recognition, sentiment analysis, and machine translation. One of the most significant advantages of LLMs is their ability to handle large amounts of text data. They can process large amounts of text data quickly and accurately, allowing them to understand and generate coherent sentences. However, they can be computationally expensive to train and maintain, and'}]

In [30]:
#predictor.delete_endpoint()